### Notebook to test setup

You can use this notebook to test commands and so forth. This notebook will not be pushed.
If it appears in git status just run `git update-index --assume-unchanged test.ipynb`.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import logging
from pprint import pprint

from fonduer import Meta, init_logging


PARALLEL = 4 # assuming a quad-core machine
ATTRIBUTE = "" # name of your db
conn_string = f'postgresql://postgres@fonduer-postgres-dev:5432/{ATTRIBUTE}'

# Configure logging for Fonduer
init_logging(log_dir="logs")


In [ ]:
# imports
from fonduer.meta import Meta
from fonduer.parser.preprocessors import HTMLDocPreprocessor
from fonduer.parser import Parser

from parser_function import extract_html_files

# path strings // path strings may need to be changed depending on the rxiv_publications-folder location
dir_to_extract = "./data"
dir_of_files_bio = "./rxiv_publications/bio"
dir_of_files_med = "./rxiv_publications/med"


def main():
    session = Meta.init(conn_string).Session()
    extract_html_files(dir_to_extract, dir_of_files_bio, dir_of_files_med)

    docs_path = dir_to_extract
    doc_preprocessor = HTMLDocPreprocessor(docs_path)
    corpus_parser = Parser(session, structural=True, lingual=True)
    corpus_parser.apply(doc_preprocessor, parallelism=PARALLEL)


if __name__ == '__main__':
    main()